#### Environment

In [47]:
import pandas as pd

teams = pd.read_csv('mastering_ncaat_teams_historical.csv')
teams = teams[teams['year']>=2009]
teams = teams[['team_id', 'kenpom_id', 'torvik_id']]
print(teams.shape)
teams.head()

(1082, 3)


,team_id,kenpom_id,torvik_id
325,2009 North Carolina,2009 North Carolina,2009 North Carolina
326,2009 Connecticut,2009 Connecticut,2009 Connecticut
327,2009 Villanova,2009 Villanova,2009 Villanova
328,2009 Michigan St.,2009 Michigan St.,2009 Michigan St.
329,2009 Louisville,2009 Louisville,2009 Louisville


#### Kenpom

In [48]:
ratings = pd.read_csv('raw_kenpom_ratings_historical.csv')
ratings['kenpom_id'] = ratings['Season'].astype(str) + ' ' + ratings['TeamName']
ratings = ratings[['kenpom_id', 'AdjEM', 'AdjOE', 'AdjDE', 'OE', 'DE', 'Tempo', 'AdjTempo']]
print('Ratings: ', ratings.shape)

roster = pd.read_csv('raw_kenpom_roster_historical.csv')
roster['kenpom_id'] = roster['Season'].astype(str) + ' ' + roster['TeamName']
roster = roster[['kenpom_id', 'HgtEff', 'Size', 'Exp', 'Bench']]
print('Roster: ', roster.shape)

Ratings:  (7950, 8)
Roster:  (6306, 5)


In [49]:
# Join the Kenpom tables for the 15 years models
print('Teams: ', teams.shape)
join_1 = pd.merge(teams, ratings, on='kenpom_id', how='inner')
print('Join Ratings: ', join_1.shape)
join_2 = pd.merge(join_1, roster, on='kenpom_id', how='inner')
print('Join Roster: ', join_2.shape)

Teams:  (1082, 3)
Join Ratings:  (1082, 10)
Join Roster:  (1082, 14)


In [50]:
kenpom_df = join_2.rename(columns={
                             'AdjEM': 'kenpom_adjem',
                             'AdjOE': 'kenpom_adjoe',
                             'AdjDE': 'kenpom_adjde',
                             'OE': 'off_eff',
                             'DE': 'def_eff',
                             'Tempo': 'tempo',
                             'AdjTempo': 'adj_tempo',
                             'HgtEff' : 'height',
                             'Size': 'size',
                             'Exp': 'experience',
                             'Bench': 'bench'
})
kenpom_df.head()

,team_id,kenpom_id,torvik_id,kenpom_adjem,kenpom_adjoe,kenpom_adjde,off_eff,def_eff,tempo,adj_tempo,height,size,experience,bench
0,2009 North Carolina,2009 North Carolina,2009 North Carolina,28.5574,122.645,94.0872,117.352,96.1566,75.9017,72.4540,1.84,77.22,2.05,27.39
1,2009 Connecticut,2009 Connecticut,2009 Connecticut,28.0401,114.597,86.5571,111.344,91.4777,67.6114,65.5863,5.72,78.25,2.20,25.51
2,2009 Villanova,2009 Villanova,2009 Villanova,21.7081,113.638,91.9303,110.170,95.8818,69.0715,66.8453,-0.05,77.05,2.04,28.99
3,2009 Michigan St.,2009 Michigan St.,2009 Michigan St.,22.7829,112.732,89.9494,108.114,94.5144,66.5629,65.9592,2.24,76.68,1.57,36.37
4,2009 Louisville,2009 Louisville,2009 Louisville,25.3676,109.217,83.8490,105.801,88.4266,69.5444,66.9670,3.00,77.06,1.72,31.11


### Torvik

In [51]:
torvik = pd.read_csv('raw_torvik_teams_historical.csv')
torvik = torvik[['torvik_id', 'adj_oe', 'adj_de', 'barthag', 'wab',
                 'efg%', 'efgd%', 'ftr', 'ftrd', 'tor', 'tord', 'orb%', 'drb%',
                 'raw_tempo', '2p%', '2p%d', '3p%', '3p%d', 'blk%', 'blked%', 
                 'ast%', 'ast%d', '3pr', '3prd', 'adj_tempo', 'ft%', 'ft%d']]
print('Torvik: ', torvik.shape)
#splits = pd.read_csv('raw_torvik_splits_historical.csv')
#splits = splits[['torvik_id', 'off_dunk_fg%', 'off_dunk_share', 'off_close2_fg%', 'off_close2_share',
#                 'off_far2_fg%', 'off_far2_share', 'off_3pt_fg%', 'off_3pt_share', 
#                 'def_dunk_fg%', 'def_dunk_share', 'def_close2_fg%', 'def_close2_share',
#                 'def_far2_fg%', 'def_far2_share', 'def_3pt_fg%', 'def_3pt_share']]
#print('Splits: ', splits.shape)

Torvik:  (5960, 27)


In [52]:
print('KenPom: ', kenpom_df.shape)
df_15year = pd.merge(kenpom_df, torvik, on='torvik_id', how='inner')
print('15 Year DF: ', df_15year.shape)

KenPom:  (1082, 14)
15 Year DF:  (1082, 40)


#### Matchups

In [53]:
games = pd.read_csv('mastering_ncaat_games_historical.csv')
games = games[games['year']>=2009]
games = games[['game_id', 'year', 'region', 'round', 'high_bracket_seed', 'high_bracket_team', 'low_bracket_seed', 'low_bracket_team', 'win']]
print(games.shape)
games.head()

(943, 9)


,game_id,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,win
315,20091101,2009,Midwest,First Round,1,2009 Louisville,16,2009 Morehead St.,1
316,20091102,2009,Midwest,First Round,8,2009 Ohio St.,9,2009 Siena,0
317,20091103,2009,Midwest,First Round,5,2009 Utah,12,2009 Arizona,0
318,20091104,2009,Midwest,First Round,4,2009 Wake Forest,13,2009 Cleveland St.,0
319,20091105,2009,Midwest,First Round,6,2009 West Virginia,11,2009 Dayton,0


In [54]:
matchups_1 = pd.merge(games, df_15year, left_on='high_bracket_team', right_on='team_id', how='inner')
matchups_1 = matchups_1.drop(columns=['team_id', 'kenpom_id'])
print('High team matchup: ', matchups_1.shape)

High team matchup:  (943, 47)


In [55]:
matchups_1.columns.to_list()

['game_id',
 'year',
 'region',
 'round',
 'high_bracket_seed',
 'high_bracket_team',
 'low_bracket_seed',
 'low_bracket_team',
 'win',
 'torvik_id',
 'kenpom_adjem',
 'kenpom_adjoe',
 'kenpom_adjde',
 'off_eff',
 'def_eff',
 'tempo',
 'adj_tempo_x',
 'height',
 'size',
 'experience',
 'bench',
 'adj_oe',
 'adj_de',
 'barthag',
 'wab',
 'efg%',
 'efgd%',
 'ftr',
 'ftrd',
 'tor',
 'tord',
 'orb%',
 'drb%',
 'raw_tempo',
 '2p%',
 '2p%d',
 '3p%',
 '3p%d',
 'blk%',
 'blked%',
 'ast%',
 'ast%d',
 '3pr',
 '3prd',
 'adj_tempo_y',
 'ft%',
 'ft%d']

In [56]:
matchups_1 = matchups_1.rename(columns={
                             'kenpom_adjem': 'high_kenpom_adjem',
                             'kenpom_adjoe': 'high_kenpom_adjoe',
                             'kenpom_adjde': 'high_kenpom_adjde',
                             'off_eff': 'high_off_eff',
                             'def_eff': 'high_def_eff',
                             'height': 'high_height',
                             'size': 'high_size',
                             'experience': 'high_experience',
                             'bench': 'high_bench',
                             'adj_oe': 'high_barthag_oe',
                             'adj_de': 'high_barthag_de',
                             'barthag': 'high_barthag',
                             'wab': 'high_wab',
                             'efg%': 'high_efg%',
                             'efgd%': 'high_efgd%',
                             'ftr': 'high_ftr',
                             'ftrd': 'high_ftrd',
                             'tor': 'high_tor',
                             'tord': 'high_tord',
                             'orb%': 'high_orb%',
                             'drb%': 'high_drb%',
                             '2p%': 'high_2p%',
                             '2p%d': 'high_2p%d',
                             '3p%': 'high_3p%',
                             '3p%d': 'high_3p%d',
                             'blk%': 'high_blk%',
                             'blked%': 'high_blked%',
                             'ast%': 'high_ast%',
                             'ast%d': 'high_ast%d',
                             '3pr': 'high_3pr',
                             '3prd': 'high_3prd',
                             'ft%': 'high_ft%',
                             'ft%d': 'high_ft%d'
})

In [57]:
matchups_1['high_kenpom_adjde'] = 130 - matchups_1['high_kenpom_adjde']
matchups_1['high_def_eff'] = 130 - matchups_1['high_def_eff']
matchups_1['high_barthag_de'] = 130 - matchups_1['high_barthag_de']
matchups_1['high_efgd%'] = 100 - matchups_1['high_efgd%']
matchups_1['high_ftrd'] = 100 - matchups_1['high_ftrd']
matchups_1['high_tord'] = 100 - matchups_1['high_tord']
matchups_1['high_drb%'] = 100 - matchups_1['high_drb%']
matchups_1['high_2p%d'] = 100 - matchups_1['high_2p%d']
matchups_1['high_3p%d'] = 100 - matchups_1['high_3p%d']
matchups_1['high_blked%'] = 100 - matchups_1['high_blked%']
matchups_1['high_ast%d'] = 100 - matchups_1['high_ast%d']
matchups_1['high_3prd'] = 100 - matchups_1['high_3prd']
matchups_1['high_ft%d'] = 100 - matchups_1['high_ft%d']

In [58]:
matchups_2 = pd.merge(matchups_1, df_15year, left_on='low_bracket_team', right_on='team_id', how='inner')
matchups_2 = matchups_2.drop(columns=['team_id', 'kenpom_id'])
print('Low team matchup: ', matchups_2.shape)

Low team matchup:  (943, 85)


In [59]:
matchups_2 = matchups_2.rename(columns={
                             'kenpom_adjem': 'low_kenpom_adjem',
                             'kenpom_adjoe': 'low_kenpom_adjoe',
                             'kenpom_adjde': 'low_kenpom_adjde',
                             'off_eff': 'low_off_eff',
                             'def_eff': 'low_def_eff',
                             'height': 'low_height',
                             'size': 'low_size',
                             'experience': 'low_experience',
                             'bench': 'low_bench',
                             'adj_oe': 'low_barthag_oe',
                             'adj_de': 'low_barthag_de',
                             'barthag': 'low_barthag',
                             'wab': 'low_wab',
                             'efg%': 'low_efg%',
                             'efgd%': 'low_efgd%',
                             'ftr': 'low_ftr',
                             'ftrd': 'low_ftrd',
                             'tor': 'low_tor',
                             'tord': 'low_tord',
                             'orb%': 'low_orb%',
                             'drb%': 'low_drb%',
                             '2p%': 'low_2p%',
                             '2p%d': 'low_2p%d',
                             '3p%': 'low_3p%',
                             '3p%d': 'low_3p%d',
                             'blk%': 'low_blk%',
                             'blked%': 'low_blked%',
                             'ast%': 'low_ast%',
                             'ast%d': 'low_ast%d',
                             '3pr': 'low_3pr',
                             '3prd': 'low_3prd',
                             'ft%': 'low_ft%',
                             'ft%d': 'low_ft%d'
})

In [60]:
matchups_2['low_kenpom_adjde'] = 130 - matchups_2['low_kenpom_adjde']
matchups_2['low_def_eff'] = 130 - matchups_2['low_def_eff']
matchups_2['low_barthag_de'] = 130 - matchups_2['low_barthag_de']
matchups_2['low_efgd%'] = 100 - matchups_2['low_efgd%']
matchups_2['low_ftrd'] = 100 - matchups_2['low_ftrd']
matchups_2['low_tord'] = 100 - matchups_2['low_tord']
matchups_2['low_drb%'] = 100 - matchups_2['low_drb%']
matchups_2['low_2p%d'] = 100 - matchups_2['low_2p%d']
matchups_2['low_3p%d'] = 100 - matchups_2['low_3p%d']
matchups_2['low_blked%'] = 100 - matchups_2['low_blked%']
matchups_2['low_ast%d'] = 100 - matchups_2['low_ast%d']
matchups_2['low_3prd'] = 100 - matchups_2['low_3prd']
matchups_2['low_ft%d'] = 100 - matchups_2['low_ft%d']

In [61]:
matchups_2.head()

,game_id,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,win,torvik_id_x,...,low_3p%d,low_blk%,low_blked%,low_ast%,low_ast%d,low_3pr,low_3prd,adj_tempo_y_y,low_ft%,low_ft%d
0,20091101,2009,Midwest,First Round,1,2009 Louisville,16,2009 Morehead St.,1,2009 Louisville,...,64.3,11.5,89.8,52.0,43.7,28.6,64.1,66.6,72.4,31.5
1,20092109,2009,Midwest,Second Round,1,2009 Louisville,9,2009 Siena,1,2009 Louisville,...,66.2,11.4,92.4,52.8,43.5,25.3,67.7,71.9,66.2,31.5
2,20091102,2009,Midwest,First Round,8,2009 Ohio St.,9,2009 Siena,0,2009 Ohio St.,...,66.2,11.4,92.4,52.8,43.5,25.3,67.7,71.9,66.2,31.5
3,20093113,2009,Midwest,Sweet 16,1,2009 Louisville,12,2009 Arizona,1,2009 Louisville,...,64.5,9.0,94.0,56.3,37.5,29.0,59.3,65.4,73.4,31.8
4,20091103,2009,Midwest,First Round,5,2009 Utah,12,2009 Arizona,0,2009 Utah,...,64.5,9.0,94.0,56.3,37.5,29.0,59.3,65.4,73.4,31.8


In [62]:
matchups_2.columns.to_list()

['game_id',
 'year',
 'region',
 'round',
 'high_bracket_seed',
 'high_bracket_team',
 'low_bracket_seed',
 'low_bracket_team',
 'win',
 'torvik_id_x',
 'high_kenpom_adjem',
 'high_kenpom_adjoe',
 'high_kenpom_adjde',
 'high_off_eff',
 'high_def_eff',
 'tempo_x',
 'adj_tempo_x_x',
 'high_height',
 'high_size',
 'high_experience',
 'high_bench',
 'high_barthag_oe',
 'high_barthag_de',
 'high_barthag',
 'high_wab',
 'high_efg%',
 'high_efgd%',
 'high_ftr',
 'high_ftrd',
 'high_tor',
 'high_tord',
 'high_orb%',
 'high_drb%',
 'raw_tempo_x',
 'high_2p%',
 'high_2p%d',
 'high_3p%',
 'high_3p%d',
 'high_blk%',
 'high_blked%',
 'high_ast%',
 'high_ast%d',
 'high_3pr',
 'high_3prd',
 'adj_tempo_y_x',
 'high_ft%',
 'high_ft%d',
 'torvik_id_y',
 'low_kenpom_adjem',
 'low_kenpom_adjoe',
 'low_kenpom_adjde',
 'low_off_eff',
 'low_def_eff',
 'tempo_y',
 'adj_tempo_x_y',
 'low_height',
 'low_size',
 'low_experience',
 'low_bench',
 'low_barthag_oe',
 'low_barthag_de',
 'low_barthag',
 'low_wab',
 'l

In [63]:
matchups_df = matchups_2[['year',
                          'region',
                          'round',
                          'high_bracket_seed',
                          'high_bracket_team',
                          'low_bracket_seed',
                          'low_bracket_team',
                          'win']]
# Efficiency
matchups_df['wab'] = matchups_2['high_wab'] - matchups_2['low_wab']
matchups_df['kenpom_adjem'] = matchups_2['high_kenpom_adjem'] - matchups_2['low_kenpom_adjem']
matchups_df['kenpom_adjoe'] = matchups_2['high_kenpom_adjoe'] - matchups_2['low_kenpom_adjde']
matchups_df['kenpom_adjde'] = matchups_2['high_kenpom_adjde'] - matchups_2['low_kenpom_adjoe']
matchups_df['barthag'] = matchups_2['high_barthag'] - matchups_2['low_barthag']
matchups_df['barthag_oe'] = matchups_2['high_barthag_oe'] - matchups_2['low_barthag_de']
matchups_df['barthag_de'] = matchups_2['high_barthag_de'] - matchups_2['low_barthag_oe']
matchups_df['off_eff'] = matchups_2['high_off_eff'] - matchups_2['low_def_eff']
matchups_df['def_eff'] = matchups_2['high_def_eff'] - matchups_2['low_off_eff']
# Four Factors Offense
matchups_df['efg%'] = matchups_2['high_efg%'] - matchups_2['low_efgd%']
matchups_df['ftr'] = matchups_2['high_ftr'] - matchups_2['low_ftrd']
matchups_df['tor'] = matchups_2['high_tor'] - matchups_2['low_tord']
matchups_df['orb%'] = matchups_2['high_orb%'] - matchups_2['low_drb%']
# Four Factors Defense
matchups_df['efgd%'] = matchups_2['high_efgd%'] - matchups_2['low_efg%']
matchups_df['ftrd'] = matchups_2['high_ftrd'] - matchups_2['low_ftr']
matchups_df['tord'] = matchups_2['high_tord'] - matchups_2['low_tor']
matchups_df['drb%'] = matchups_2['high_drb%'] - matchups_2['low_orb%']
# Traditional Stats Offense
matchups_df['2p%'] = matchups_2['high_2p%'] - matchups_2['low_2p%d']
matchups_df['3p%'] = matchups_2['high_3p%'] - matchups_2['low_3p%d']
matchups_df['blked%'] = matchups_2['high_blked%'] - matchups_2['low_blk%']
matchups_df['ast%'] = matchups_2['high_ast%'] - matchups_2['low_ast%d']
matchups_df['3pr'] = matchups_2['high_3pr'] - matchups_2['low_3prd']
matchups_df['ft%'] = matchups_2['high_ft%'] - matchups_2['low_ft%d']
# Traditional Stats Defense
matchups_df['2p%d'] = matchups_2['high_2p%d'] - matchups_2['low_2p%']
matchups_df['3p%d'] = matchups_2['high_3p%d'] - matchups_2['low_3p%']
matchups_df['blk%'] = matchups_2['high_blk%'] - matchups_2['low_blked%']
matchups_df['ast%d'] = matchups_2['high_ast%d'] - matchups_2['low_ast%']
matchups_df['3prd'] = matchups_2['high_3prd'] - matchups_2['low_3pr']
matchups_df['ft%d'] = matchups_2['high_ft%d'] - matchups_2['low_ft%']
# Miscellaneous Stats
matchups_df['height'] = matchups_2['high_height'] - matchups_2['low_height']
matchups_df['size'] = matchups_2['high_size'] - matchups_2['low_size']
matchups_df['experience'] = matchups_2['high_experience'] - matchups_2['low_experience']
matchups_df['bench'] = matchups_2['high_bench'] - matchups_2['low_bench']

/var/folders/nd/7h3lcr8d2cjbxfmfqghczqz40000gn/T/ipykernel_95932/940464427.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchups_df['wab'] = matchups_2['high_wab'] - matchups_2['low_wab']
/var/folders/nd/7h3lcr8d2cjbxfmfqghczqz40000gn/T/ipykernel_95932/940464427.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matchups_df['kenpom_adjem'] = matchups_2['high_kenpom_adjem'] - matchups_2['low_kenpom_adjem']
/var/folders/nd/7h3lcr8d2cjbxfmfqghczqz40000gn/T/ipykernel_95932/940464427.py:12: SettingWith

In [64]:
print(matchups_df.shape)
matchups_df.head()

(943, 41)


,year,region,round,high_bracket_seed,high_bracket_team,low_bracket_seed,low_bracket_team,win,wab,kenpom_adjem,...,2p%d,3p%d,blk%,ast%d,3prd,ft%d,height,size,experience,bench
0,2009,Midwest,First Round,1,2009 Louisville,16,2009 Morehead St.,1,17.203930,25.384675,...,8.9,36.9,-74.1,-4.6,41.0,-40.3,7.12,1.20,0.13,7.28
1,2009,Midwest,Second Round,1,2009 Louisville,9,2009 Siena,1,7.799197,13.693900,...,4.8,36.1,-76.7,-5.4,44.3,-34.1,5.17,0.66,-0.06,10.87
2,2009,Midwest,First Round,8,2009 Ohio St.,9,2009 Siena,0,2.952748,4.791000,...,3.1,31.3,-76.1,-13.6,33.8,-35.5,4.47,1.73,-0.78,4.58
3,2009,Midwest,Sweet 16,1,2009 Louisville,12,2009 Arizona,1,7.341754,9.354300,...,6.1,30.1,-78.3,-8.9,40.6,-41.3,0.52,0.20,0.29,13.40
4,2009,Midwest,First Round,5,2009 Utah,12,2009 Arizona,0,2.034709,3.024800,...,7.4,25.5,-81.8,-3.3,44.4,-45.3,2.43,1.30,1.00,3.99


In [65]:
matchups_df.columns.to_list()

['year',
 'region',
 'round',
 'high_bracket_seed',
 'high_bracket_team',
 'low_bracket_seed',
 'low_bracket_team',
 'win',
 'wab',
 'kenpom_adjem',
 'kenpom_adjoe',
 'kenpom_adjde',
 'barthag',
 'barthag_oe',
 'barthag_de',
 'off_eff',
 'def_eff',
 'efg%',
 'ftr',
 'tor',
 'orb%',
 'efgd%',
 'ftrd',
 'tord',
 'drb%',
 '2p%',
 '3p%',
 'blked%',
 'ast%',
 '3pr',
 'ft%',
 '2p%d',
 '3p%d',
 'blk%',
 'ast%d',
 '3prd',
 'ft%d',
 'height',
 'size',
 'experience',
 'bench']

In [66]:
matchups_df.to_csv('matchups_15.csv')